In [ ]:
# Install PyTorch
!pip install torch torchvision torchaudio

# Install PyTorch Geometric and dependencies
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-2.0.0+cpu.html

# Verify Installation
# Import necessary libraries
import torch_geometric
print("PyTorch Geometric version:", torch_geometric.__version__)

Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
PyTorch Geometric version: 2.6.1


In [ ]:
# Import necessary libraries
import tensorflow as tf
# Import necessary libraries
import numpy as np
# Import necessary libraries
import tensorflow.keras.backend as K
from tensorflow.keras import layers
from tensorflow.keras.models import Model
# Import necessary libraries
import tensorflow.keras.optimizers as optimizers
from scipy.sparse import coo_matrix
# Import necessary libraries
import pandas as pd
from scipy.spatial.distance import cdist
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_scipy_sparse_matrix

In [ ]:

#First version of FastGCNLayer
class FastGCNLayer(tf.keras.layers.Layer):
# Function definition
    def __init__(self, output_dim, activation=None, dropout_rate=0.0, **kwargs):
        super(FastGCNLayer, self).__init__(**kwargs)
        self.output_dim = output_dim
        self.activation = activation
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

# Function definition
    def build(self, input_shape):
        self.kernel = self.add_weight(
            shape=(input_shape[-1], self.output_dim),
            initializer="glorot_uniform",
            trainable=True,
        )
        super(FastGCNLayer, self).build(input_shape)

# Function definition
    def call(self, features, adj, sampled_nodes, training=False):
        # Convert sampled_nodes to int64 to match adj.indices data type
        sampled_nodes = tf.cast(sampled_nodes, dtype=tf.int64)

# Loop execution
        # Filter adjacency matrix rows and columns for sampled nodes
        row_mask = tf.reduce_any(adj.indices[:, 0:1] == sampled_nodes, axis=1)
        col_mask = tf.reduce_any(adj.indices[:, 1:2] == sampled_nodes, axis=1)
        sampled_adj_indices = tf.boolean_mask(adj.indices, tf.logical_and(row_mask, col_mask))
        sampled_adj_values = tf.boolean_mask(adj.values, tf.logical_and(row_mask, col_mask))

        # Remap node indices to the range 0 to len(sampled_nodes) - 1
        unique_nodes, remapped_indices = tf.unique(tf.reshape(sampled_adj_indices, [-1]))

        # Explicitly cast tf.range to int64 to match expected data type
        node_map = tf.lookup.StaticHashTable(
            tf.lookup.KeyValueTensorInitializer(unique_nodes, tf.range(tf.size(unique_nodes), dtype=tf.int64)), # Cast to int64
            -1
        )
        remapped_indices = tf.reshape(node_map.lookup(tf.reshape(sampled_adj_indices, [-1])), tf.shape(sampled_adj_indices))

        # Get the actual number of valid sampled nodes
        num_valid_nodes = tf.shape(unique_nodes)[0]

        sampled_adj = tf.sparse.SparseTensor(
            indices=remapped_indices, #Use remapped indices here
            values=sampled_adj_values,
            # Use num_valid_nodes instead of len(sampled_nodes)
            dense_shape=[num_valid_nodes, num_valid_nodes]
        )
        sampled_adj = tf.sparse.reorder(sampled_adj)

# Loop execution
        # Extract features for sampled nodes
        # Remap sampled_nodes to the new range (0 to num_samples - 1)
# Loop execution
        # This is crucial for the second layer
        sampled_nodes_remapped = node_map.lookup(sampled_nodes)

        # Filter out invalid indices (-1)
        valid_indices_mask = tf.where(sampled_nodes_remapped >= 0)
        sampled_nodes_remapped = tf.gather_nd(sampled_nodes_remapped, valid_indices_mask)

# Loop execution
        # Gather features for valid indices only
        sampled_features = tf.gather(features, sampled_nodes_remapped, validate_indices=False)

        # Perform sparse neighborhood aggregation
        aggregated_features = tf.sparse.sparse_dense_matmul(sampled_adj, sampled_features)
        aggregated_features = tf.matmul(aggregated_features, self.kernel)

        # Apply activation and dropout
# Conditional check
        if self.activation:
            aggregated_features = self.activation(aggregated_features)
        return self.dropout(aggregated_features, training=training)


In [ ]:
'''
# Loop execution
First Version of train function ( the one originally used for CORA )



# Training Function
# Function definition
def train(model, features, adj, labels, train_mask, num_epochs, num_samples, learning_rate):
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Loop execution
    for epoch in range(num_epochs):
        with tf.GradientTape() as tape:
            # Importance sampling
            sampled_nodes = importance_sampling(adj, num_samples)

            # Forward pass
            logits = model(features, adj, sampled_nodes, training=True)

            # Ensure valid_sampled_nodes are less than the size of logits
            valid_sampled_indices = tf.where(sampled_nodes < tf.shape(logits)[0])[:, 0]

            # Clip valid_sampled_indices to be within the range of logits
            # Cast tf.shape(logits)[0] - 1 to int64 to match valid_sampled_indices
            valid_sampled_indices = tf.clip_by_value(valid_sampled_indices, 0, tf.cast(tf.shape(logits)[0] - 1, tf.int64)) # Changed to int64

# Loop execution
            # Gather labels and logits for valid sampled nodes only
            sampled_labels = tf.gather(labels, tf.gather(sampled_nodes, valid_sampled_indices))
            sampled_logits = tf.gather(logits, valid_sampled_indices)

            # Compute loss
            loss = loss_fn(sampled_labels, sampled_logits)

            # Backward pass
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

           # print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.numpy():.4f}")
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.numpy().item():.4f}")
'''

'\nFirst Version of train function ( the one originally used for CORA )\n\n\n\n# Training Function\ndef train(model, features, adj, labels, train_mask, num_epochs, num_samples, learning_rate):\n    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)\n    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)\n\n    for epoch in range(num_epochs):\n        with tf.GradientTape() as tape:\n            # Importance sampling\n            sampled_nodes = importance_sampling(adj, num_samples)\n\n            # Forward pass\n            logits = model(features, adj, sampled_nodes, training=True)\n\n            # Ensure valid_sampled_nodes are less than the size of logits\n            valid_sampled_indices = tf.where(sampled_nodes < tf.shape(logits)[0])[:, 0]\n\n            # Clip valid_sampled_indices to be within the range of logits\n            # Cast tf.shape(logits)[0] - 1 to int64 to match valid_sampled_indices\n            valid_sampled_indices = tf.clip_

In [ ]:

'''
  Second version of FastGCNLayer with this changes:

# Loop execution
  Added layer normalization for better training stability
  Improved adjacency matrix normalization
  Added bias term option
  More efficient sparse matrix operations

  DON'T USE DOES NOT WORKING
'''
'''
class FastGCNLayer(tf.keras.layers.Layer):
# Function definition
    def __init__(self, output_dim, activation=None, dropout_rate=0.0, **kwargs):
        super(FastGCNLayer, self).__init__(**kwargs)
        self.output_dim = output_dim
        self.activation = activation
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

# Function definition
    def build(self, input_shape):
        self.kernel = self.add_weight(
            shape=(input_shape[-1], self.output_dim),
            initializer="glorot_uniform",
            trainable=True,
        )
        super(FastGCNLayer, self).build(input_shape)

# Function definition
    def call(self, features, adj, sampled_nodes, training=False):
        # Convert sampled_nodes to int64
        sampled_nodes = tf.cast(sampled_nodes, dtype=tf.int64)

        # First transform the features using the kernel
        transformed_features = tf.matmul(features, self.kernel)

# Loop execution
        # Create a mask for the sampled nodes but using the shape of transformed_features
        batch_size = tf.shape(transformed_features)[0] #changed here
        num_nodes = tf.shape(transformed_features)[1] #changed here

# Loop execution
        # Create a sparse mask matrix for sampling using new batch_size and num_nodes values
        indices = tf.stack([
            tf.range(batch_size, dtype=tf.int64),
            tf.cast(sampled_nodes, tf.int64)
        ], axis=1)

        values = tf.ones([batch_size], dtype=tf.float32)
        mask = tf.sparse.SparseTensor(
            indices=indices,
            values=values,
            dense_shape=[batch_size, num_nodes] #using new values here
        )

        # Sample the adjacency matrix using the mask
        sampled_features = tf.sparse.sparse_dense_matmul(mask, transformed_features)

        # Apply activation and dropout
# Conditional check
        if self.activation:
            sampled_features = self.activation(sampled_features)

        return self.dropout(sampled_features, training=training)
'''

'\nclass FastGCNLayer(tf.keras.layers.Layer):\n    def __init__(self, output_dim, activation=None, dropout_rate=0.0, **kwargs):\n        super(FastGCNLayer, self).__init__(**kwargs)\n        self.output_dim = output_dim\n        self.activation = activation\n        self.dropout = tf.keras.layers.Dropout(dropout_rate)\n\n    def build(self, input_shape):\n        self.kernel = self.add_weight(\n            shape=(input_shape[-1], self.output_dim),\n            initializer="glorot_uniform",\n            trainable=True,\n        )\n        super(FastGCNLayer, self).build(input_shape)\n\n    def call(self, features, adj, sampled_nodes, training=False):\n        # Convert sampled_nodes to int64\n        sampled_nodes = tf.cast(sampled_nodes, dtype=tf.int64)\n\n        # First transform the features using the kernel\n        transformed_features = tf.matmul(features, self.kernel)\n\n        # Create a mask for the sampled nodes but using the shape of transformed_features\n        batch_size

In [ ]:
# FastGCN Model
class FastGCN(tf.keras.Model):
# Function definition
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_rate=0.5):
        super(FastGCN, self).__init__()
        self.layer1 = FastGCNLayer(hidden_dim, activation=tf.nn.relu, dropout_rate=dropout_rate)
        self.layer2 = FastGCNLayer(output_dim, activation=None, dropout_rate=dropout_rate)

# Function definition
    def call(self, features, adj, sampled_nodes, training=False):
        x = self.layer1(features, adj, sampled_nodes, training=training)
        x = self.layer2(x, adj, sampled_nodes, training=training)
        return x

In [ ]:
# Importance Sampling Function
# Function definition
def importance_sampling(adj, num_samples):
    degrees = tf.sparse.reduce_sum(adj, axis=1) + 1e-5  # Avoid division by zero
    probabilities = degrees / tf.reduce_sum(degrees)
    sampled_nodes = np.random.choice(adj.shape[0], num_samples, replace=False, p=probabilities.numpy())
    return tf.constant(sampled_nodes, dtype=tf.int32)


In [ ]:

#train used with mnist dataset and PubMed
# Function definition
def train(model, features, adj, labels, train_mask, num_epochs, num_samples, learning_rate):

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    # Ensure num_samples is not larger than the number of nodes
    num_samples = min(num_samples, features.shape[0])

# Loop execution
    for epoch in range(num_epochs):
        with tf.GradientTape() as tape:
            # Importance sampling
            sampled_nodes = importance_sampling(adj, num_samples)

            # Forward pass
            logits = model(features, adj, sampled_nodes, training=True)

            # Ensure we only use valid indices
            valid_mask = sampled_nodes < tf.shape(logits)[0]
            valid_sampled_nodes = tf.boolean_mask(sampled_nodes, valid_mask)

# Loop execution
            # Gather labels and logits for valid sampled nodes
            sampled_labels = tf.gather(labels, valid_sampled_nodes)
            sampled_logits = tf.gather(logits, tf.range(tf.shape(valid_sampled_nodes)[0]))

            # Compute loss and reduce to scalar
            loss = tf.reduce_mean(loss_fn(sampled_labels, sampled_logits))

# Conditional check
        # Check if loss is valid before proceeding
# Conditional check
        if tf.math.is_nan(loss):
            print(f"Epoch {epoch+1}/{num_epochs}, Warning: NaN loss detected")
            continue

        # Compute and clip gradients
        gradients = tape.gradient(loss, model.trainable_variables)
        gradients, _ = tf.clip_by_global_norm(gradients, 5.0)

# Loop execution
        # Check for valid gradients
# Loop execution
        if any(tf.reduce_any(tf.math.is_nan(g)) if g is not None else False for g in gradients):
            print(f"Epoch {epoch+1}/{num_epochs}, Warning: NaN gradients detected")
            continue

        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        # Convert loss to Python scalar before printing
        try:
            loss_value = float(loss.numpy())
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss_value:.4f}")
        except:
            print(f"Epoch {epoch+1}/{num_epochs}, Error printing loss")


In [ ]:
#batched version
# Function definition
def create_graph_structure(features, k=15, metric='euclidean', batch_size=1000):

# Loop execution
    """Create k-nearest neighbors graph for MNIST with batched processing"""
    print("Creating MNIST graph structure...")
    num_samples = len(features)
    edges = []

# Loop execution
    for i in range(0, num_samples, batch_size):
        end = min(i + batch_size, num_samples)
        batch_features = features[i:end]

# Loop execution
        # Compute distances for current batch
        distances = cdist(batch_features, features, metric=metric)

        # Find k nearest neighbors
        nearest = np.argpartition(distances, k+1, axis=1)[:, :k+1]

# Loop execution
        # Create edges (both directions for undirected graph)
# Loop execution
        for idx, neighbors in enumerate(nearest):
            global_idx = i + idx
            valid_neighbors = neighbors[neighbors != global_idx][:k]
# Loop execution
            edges.extend([(global_idx, n) for n in valid_neighbors])
# Loop execution
            edges.extend([(n, global_idx) for n in valid_neighbors])

    # Create sparse adjacency matrix
    edges = np.array(edges)
    values = np.ones(len(edges), dtype=np.float32)
    adj_matrix = coo_matrix(
        (values, (edges[:, 0], edges[:, 1])),
        shape=(num_samples, num_samples)
    )

    return tf.sparse.reorder(tf.sparse.SparseTensor(
        indices=np.vstack((adj_matrix.row, adj_matrix.col)).T,
        values=adj_matrix.data,
        dense_shape=adj_matrix.shape
    ))



In [ ]:
# Function definition
def load_mnist_file(filepath):

    try:
        data = pd.read_csv(filepath)

        '''
        labels = data['label'].values
        features = data.drop('label', axis=1).values.astype(np.float32)
        '''

        labels = data.iloc[:, 0].values  # Estrai la prima colonna come etichette
        features = data.iloc[:, 1:].values.astype(np.float32)  # Resto delle colonne sono le features

        features = features / 255.0  # Normalize pixel values to [0, 1]

        return features, labels

    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return None, None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

In [ ]:
# Loop execution
# Function to check tensor properties (useful for debugging)
# Function definition
def print_tensor_info(tensor, name):
    print(f"\n{name}:")
    print(f"  Shape: {tensor.shape}")
    print(f"  Dtype: {tensor.dtype}")
    print(f"  Min: {tf.reduce_min(tensor)}")
    print(f"  Max: {tf.reduce_max(tensor)}")

# Modified evaluation helper function
# Function definition
def evaluate_model(model, features, adj, labels, num_samples):
    sampled_nodes = importance_sampling(adj, num_samples)
    logits = model(features, adj, sampled_nodes, training=False)

    valid_mask = sampled_nodes < tf.shape(logits)[0]
    valid_sampled_nodes = tf.cast(tf.boolean_mask(sampled_nodes, valid_mask), tf.int64)

    predictions = tf.cast(
        tf.argmax(tf.gather(logits, tf.range(tf.shape(valid_sampled_nodes)[0])), axis=1),
        tf.int64
    )
    labels_sampled = tf.gather(labels, valid_sampled_nodes)

    accuracy = tf.reduce_mean(
        tf.cast(tf.equal(predictions, labels_sampled), tf.float32)
    )

    return accuracy, predictions, labels_sampled

In [ ]:
 # Load data

train_features, train_labels = load_mnist_file("/content/sample_data/mnist_train_small.csv")

# Conditional check
if train_features is not None and train_labels is not None:
  print("trainData loaded successfully!")
  print("TainFeatures shape:", train_features.shape)
  print("trainLabels shape:", train_labels.shape)

test_features, test_labels = load_mnist_file("/content/sample_data/mnist_test.csv")

# Conditional check
if test_features is not None and test_labels is not None:
  print("testData loaded successfully!")
  print("TestFeatures shape:", test_features.shape)
  print("TestLabels shape:", test_labels.shape)

# Convert to TensorFlow tensors with explicit types
train_features = tf.convert_to_tensor(train_features, dtype=tf.float32)
train_labels = tf.convert_to_tensor(train_labels, dtype=tf.int64)  # Changed to int64
test_features = tf.convert_to_tensor(test_features, dtype=tf.float32)
test_labels = tf.convert_to_tensor(test_labels, dtype=tf.int64)    # Changed to int64

trainData loaded successfully!
TainFeatures shape: (19999, 784)
trainLabels shape: (19999,)
testData loaded successfully!
TestFeatures shape: (9999, 784)
TestLabels shape: (9999,)


In [ ]:
# Create graph structure using k-nearest neighbors
print("Creating training graph structure...")
train_adj = create_graph_structure(train_features.numpy())

# Loop execution
# Debugging information for training graph
print("\nTraining Graph Debug Info:")
print(f"Number of nodes: {train_adj.shape[0]}")
print(f"Number of edges: {train_adj.indices.shape[0]} (counted as directed edges)")
print(f"Adjacency matrix density: {train_adj.indices.shape[0] / (train_adj.shape[0] ** 2):.6f}")

print("\nCreating testing graph structure...")
test_adj = create_graph_structure(test_features.numpy())

# Loop execution
# Debugging information for testing graph
print("\nTesting Graph Debug Info:")
print(f"Number of nodes: {test_adj.shape[0]}")
print(f"Number of edges: {test_adj.indices.shape[0]} (counted as directed edges)")
print(f"Adjacency matrix density: {test_adj.indices.shape[0] / (test_adj.shape[0] ** 2):.6f}")


Creating training graph structure...
Creating MNIST graph structure...

Training Graph Debug Info:
Number of nodes: 19999
Number of edges: 599970 (counted as directed edges)
Adjacency matrix density: 0.001500

Creating testing graph structure...
Creating MNIST graph structure...

Testing Graph Debug Info:
Number of nodes: 9999
Number of edges: 299970 (counted as directed edges)
Adjacency matrix density: 0.003000


In [ ]:
# Model parameters
input_dim = train_features.shape[1]
hidden_dim = 16
output_dim = 10

# Initialize model
model = FastGCN(input_dim, hidden_dim, output_dim, dropout_rate=0.3)

# Training parameters
num_samples = min(500, train_features.shape[0])
num_epochs = 50
learning_rate = 0.0005

print(f"Starting training with {num_samples} samples per epoch...")

# Train the model
train(
    model=model,
    features=train_features,
    adj=train_adj,
    labels=train_labels,
    train_mask=np.ones(len(train_features), dtype=bool),
    num_epochs=num_epochs,
    num_samples=num_samples,
    learning_rate=learning_rate
)

Starting training with 500 samples per epoch...
Epoch 1/50, Loss: 4.3535
Epoch 2/50, Loss: 4.4343
Epoch 3/50, Loss: 2.1507
Epoch 4/50, Loss: 2.7139
Epoch 5/50, Loss: 3.7070
Epoch 6/50, Loss: 1.9476
Epoch 7/50, Loss: 2.6127
Epoch 8/50, Loss: 2.2940
Epoch 9/50, Loss: 3.0446
Epoch 10/50, Loss: 3.1528
Epoch 11/50, Loss: 2.7352
Epoch 12/50, Loss: 2.4396
Epoch 13/50, Loss: 2.4598
Epoch 14/50, Loss: 3.3334
Epoch 15/50, Loss: 2.3016
Epoch 16/50, Loss: 2.4532
Epoch 17/50, Loss: 4.0046
Epoch 18/50, Loss: 2.3977
Epoch 19/50, Loss: 2.3872
Epoch 20/50, Loss: 2.3758
Epoch 21/50, Loss: 2.3297
Epoch 22/50, Loss: 3.5467


KeyboardInterrupt: 

In [ ]:

# Evaluation
print("\nEvaluating model...")
try:
  sampled_nodes_test = importance_sampling(test_adj, num_samples)
  test_logits = model(test_features, test_adj, sampled_nodes_test, training=False)

  # Ensure predictions and labels have the same shape and type
  valid_mask = sampled_nodes_test < tf.shape(test_logits)[0]
  valid_sampled_nodes = tf.boolean_mask(sampled_nodes_test, valid_mask)

  # Make sure all tensors are the same type (int64)
  valid_sampled_nodes = tf.cast(valid_sampled_nodes, tf.int64)
  test_predictions = tf.cast(
      tf.argmax(tf.gather(test_logits, tf.range(tf.shape(valid_sampled_nodes)[0])), axis=1),
      tf.int64
      )
  test_labels_sampled = tf.gather(test_labels, valid_sampled_nodes)

  # Calculate accuracy
  test_accuracy = tf.reduce_mean(
      tf.cast(tf.equal(test_predictions, test_labels_sampled), tf.float32)
      )

  print(f"\nTest Accuracy: {test_accuracy:.4f}")


except Exception as e:
  print(f"Error during evaluation: {str(e)}")
  print("Debug information:")
  print(f"Test logits shape: {test_logits.shape}")
  print(f"Valid sampled nodes shape: {valid_sampled_nodes.shape}")
  print(f"Test predictions shape: {test_predictions.shape}")
  print(f"Test labels sampled shape: {test_labels_sampled.shape}")
  raise

In [ ]:
# Load the PubMed dataset
dataset = Planetoid(root='/tmp/PubMed', name='PubMed')
data = dataset[0]  # Get the data object

# Loop execution
# Convert features and labels to NumPy for TensorFlow compatibility
features = data.x.numpy().astype(np.float32)  # Feature matrix
labels = data.y.numpy().astype(np.int64)  # Node labels

# Convert adjacency matrix to SciPy sparse matrix
adj_matrix = to_scipy_sparse_matrix(data.edge_index).tocoo()

# Convert adjacency matrix to TensorFlow SparseTensor
adj_tensor = tf.sparse.SparseTensor(
    indices=np.vstack((adj_matrix.row, adj_matrix.col)).T,
    values=adj_matrix.data,
    dense_shape=adj_matrix.shape
)
adj_tensor = tf.sparse.reorder(adj_tensor)

# Prepare train, validation, and test masks
num_nodes = features.shape[0]

train_mask = np.zeros(num_nodes, dtype=bool)
train_mask[data.train_mask.numpy()] = True

val_mask = np.zeros(num_nodes, dtype=bool)
val_mask[data.val_mask.numpy()] = True

test_mask = np.zeros(num_nodes, dtype=bool)
test_mask[data.test_mask.numpy()] = True

# Debugging information
print("Dataset:", dataset)
print("Number of nodes:", num_nodes)
print("Feature matrix shape:", features.shape)
print("Number of edges:", adj_matrix.nnz)
print("Number of classes:", np.max(labels) + 1)
print("Train nodes:", np.sum(train_mask))
print("Validation nodes:", np.sum(val_mask))
print("Test nodes:", np.sum(test_mask))


In [ ]:
# Model parameters
input_dim = features.shape[1]
hidden_dim = 128
output_dim = labels.max() + 1

# Initialize the FastGCN model
model = FastGCN(input_dim, hidden_dim, output_dim, dropout_rate=0.5)

# Training parameters
num_samples = 500  # Number of nodes sampled per epoch
num_epochs = 50
learning_rate = 0.001

# Train the model
train(
    model=model,
    features=features,
    adj=adj_tensor,
    labels=labels,
    train_mask=train_mask,
    num_epochs=num_epochs,
    num_samples=num_samples,
    learning_rate=learning_rate
)

# Evaluate the model on the test set
test_accuracy, _, _ = evaluate_model(
    model=model,
    features=features,
    adj=adj_tensor,
    labels=labels,
    num_samples=np.sum(test_mask)
)
print(f"Test Accuracy on PubMed: {test_accuracy:.4f}")

In [ ]:
# Function definition
def load_cora():
    num_nodes = 2708
    num_features = 1433
    num_classes = 7

    # Simulate feature matrix and labels
    features = np.random.rand(num_nodes, num_features).astype(np.float32)
    labels = np.random.randint(0, num_classes, size=(num_nodes,), dtype=np.int64)

    # Simulate adjacency matrix (sparse graph structure)
    row = np.random.randint(0, num_nodes, size=10000)
    col = np.random.randint(0, num_nodes, size=10000)
    data = np.ones(len(row), dtype=np.float32)
    adjacency = coo_matrix((data, (row, col)), shape=(num_nodes, num_nodes))

# Loop execution
    # Define train mask (e.g., first 140 nodes for training)
    train_mask = np.zeros(num_nodes, dtype=bool)
    train_mask[:140] = True

    return features, labels, adjacency, train_mask

# Load data
features, labels, adjacency, train_mask = load_cora()

# Convert adjacency matrix to TensorFlow SparseTensor
adj_tensor = tf.sparse.SparseTensor(
    indices=np.vstack((adjacency.row, adjacency.col)).T,
    values=adjacency.data,
    dense_shape=adjacency.shape
)
adj_tensor = tf.sparse.reorder(adj_tensor)

# Model parameters
input_dim = features.shape[1]
hidden_dim = 16
output_dim = labels.max() + 1

# Initialize and train the model
model = FastGCN(input_dim, hidden_dim, output_dim, dropout_rate=0.5)
train(model, features, adj_tensor, labels, train_mask, num_epochs=50, num_samples=500, learning_rate=0.001)

# Conditional check
# Evaluate the model on the entire dataset (or a subset if needed)
accuracy, _, _ = evaluate_model(
    model = model,
    features = features,
    adj = adj_tensor,
    labels = labels,
    num_samples=features.shape[0])

print(f"Test Accuracy on CORA: {accuracy.numpy():.4f}")

In [ ]:
# Load the CiteSeer dataset
dataset = Planetoid(root='/tmp/CiteSeer', name='CiteSeer')
data = dataset[0]  # Get the data object

# Loop execution
# Convert features and labels to NumPy for TensorFlow compatibility
features = data.x.numpy().astype(np.float32)  # Feature matrix
labels = data.y.numpy().astype(np.int64)  # Node labels

# Convert adjacency matrix to SciPy sparse matrix
adj_matrix = to_scipy_sparse_matrix(data.edge_index).tocoo()

# Convert adjacency matrix to TensorFlow SparseTensor
adj_tensor = tf.sparse.SparseTensor(
    indices=np.vstack((adj_matrix.row, adj_matrix.col)).T,
    values=adj_matrix.data,
    dense_shape=adj_matrix.shape
)
adj_tensor = tf.sparse.reorder(adj_tensor)

# Prepare train, validation, and test masks
num_nodes = features.shape[0]

train_mask = np.zeros(num_nodes, dtype=bool)
train_mask[data.train_mask.numpy()] = True

val_mask = np.zeros(num_nodes, dtype=bool)
val_mask[data.val_mask.numpy()] = True

test_mask = np.zeros(num_nodes, dtype=bool)
test_mask[data.test_mask.numpy()] = True

# Debugging information
print("Dataset:", dataset)
print("Number of nodes:", num_nodes)
print("Feature matrix shape:", features.shape)
print("Number of edges:", adj_matrix.nnz)
print("Number of classes:", np.max(labels) + 1)
print("Train nodes:", np.sum(train_mask))
print("Validation nodes:", np.sum(val_mask))
print("Test nodes:", np.sum(test_mask))

# Model parameters
input_dim = features.shape[1]
# Conditional check
hidden_dim = 128  # Adjust hidden dimension if needed
output_dim = labels.max() + 1

# Initialize the FastGCN model
model = FastGCN(input_dim, hidden_dim, output_dim, dropout_rate=0.5)

# Training parameters
num_samples = 500  # Number of nodes sampled per epoch
num_epochs = 50
learning_rate = 0.001

# Train the model
train(
    model=model,
    features=features,
    adj=adj_tensor,
    labels=labels,
    train_mask=train_mask,
    num_epochs=num_epochs,
    num_samples=num_samples,
    learning_rate=learning_rate
)

# Evaluate the model on the test set
test_accuracy, _, _ = evaluate_model(
    model=model,
    features=features,
    adj=adj_tensor,
    labels=labels,
    num_samples=np.sum(test_mask)
)
print(f"Test Accuracy on CiteSeer: {test_accuracy:.4f}")